In [1]:
import sys
import numpy as np
sys.path.insert(0, '/home/david/Institut/rydanalysis/')
from rydanalysis import *
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from scipy import interpolate
mpl.rc('image', cmap='viridis')
sns.set_style("whitegrid")
from lmfit import Model,Parameters
from scipy import ndimage
import xarray as xr
from tqdm import tqdm

In [2]:
def fit22dGaussian(image):
    model_twoG = Model(two_Gaussian)

    params = Parameters()

    params = model_twoG.make_params()



    # cloud distribution
    params.add('amp1',value=0.1)
    params.add('cen_y1',value=128*binning_scale,min=100*binning_scale,max=200*binning_scale)
    params.add('cen_x1',value=43*binning_scale,min=20*binning_scale,max=60*binning_scale)
    params.add('sig_x1',value=27*binning_scale,min=10*binning_scale,max=100*binning_scale)
    params.add('sig_y1',value=66*binning_scale,min=10*binning_scale,max=100*binning_scale)
    params.add('theta1',value=0,min=-np.pi/10,max=np.pi/10)

    # EIT/Autler-Townes-dip
    params.add('amp2',value=0.00,vary=True)
    params.add('cen_y2',value=150*binning_scale,min=140*binning_scale,max=160*binning_scale,vary=True)
    params.add('cen_x2',value=39*binning_scale,min=36*binning_scale,max=48*binning_scale,vary=True)
    params.add('sig_x2',value=5*binning_scale,min=3*binning_scale,max=8*binning_scale,vary=False)
    #params.add('sigma_y2',value=10,min=3,max=15)
    params.add('sig_y2',expr='sig_x2',vary=False)
    params.add('theta2',value=0.0000001,min=0,max=np.pi,vary=False)

    # offset
    params.add('offset',value=0.000001,vary=False)
    model = model_twoG
    
    shape = image.shape
    x,y = np.mgrid[0:shape[0],0:shape[1]]
    
    image_flat=image.flatten() 
    #weights_flat=image_weights.flatten()
    out = model.fit(image_flat,params,xy_mesh=(x,y))
    return out

In [3]:
#od = ndimage.gaussian_filter(od, 4, order=0, output=None, mode='constant', cval=0.0, truncate=4.0)

def fit2dGaussian(image):
    #image = ndimage.gaussian_filter(image, 8, order=0, output=None, mode='constant', cval=0.0, truncate=4.0)
    fit = Fit2dGaussian(image)
    params = Parameters()#fit.params
    params.add('amp',value=0.1)
    
    params.add('cen_x',value=50,min=45,max=55)
    params.add('cen_y',value=165,min=155,max=175)
    
    params.add('sig_x',value=51,min=30,max=100)
    params.add('sig_y',value=160,min=140,max=180)
    
    params.add('theta',vary=False,value=0,min=0,max=np.pi)
    #fit.params = fit.guess(image)
    #fit.params = restrict_to_init(fit.params,dev=0.2)
    #fit.params['amp'].max = fit.params['amp'].value*(1+0.5)
    #fit.params['amp'].min = fit.params['amp'].value*(1-0.5)
    #fit.params['cen_x'].max = fit.params['amp'].value*(1+0.1)
    #fit.params['cen_x'].min = fit.params['amp'].value*(1-0.1)
    #fit.params['cen_y'].max = fit.params['amp'].value*(1+0.1)
    #fit.params['cen_y'].min = fit.params['amp'].value*(1-0.1)
    #fit.params['offset'].max = fit.params['amp'].value*(1+2)
    params.add('offset')
    fit.params=params
    fit.params['offset'].vary = False
    fit.params['offset'].value = 0
    
    fit.fit_data()
    return xr.DataArray(fit)

In [4]:
seq = ExpSequence("/home/david/Institut/2019_IEI_new/July/19/Linescan_lowOMEGA_lowDENSITY")
variables= seq.variables.copy()
for i,shot in tqdm(enumerate(seq.iter_single_shots())):
    image=shot.image
    absorb = calc_absorbtion_image(image)
    absorb = crop_image(absorb,xslice=slice(10,90),yslice=slice(10,380))
    od = absorbtion_to_OD(absorb)
    od =  nn_replace_nonfinite(od,~np.isfinite(od))
    shot.optical_density = od
data = seq.variables.copy()

0it [00:00, ?it/s]/home/david/Institut/rydanalysis/rydanalysis/single_shot/image_processing.py:20: RuntimeWarning: divide by zero encountered in true_divide
  trans = atoms/light
/home/david/Institut/rydanalysis/rydanalysis/single_shot/image_processing.py:20: RuntimeWarning: invalid value encountered in true_divide
  trans = atoms/light
/home/david/Institut/rydanalysis/rydanalysis/single_shot/image_processing.py:9: RuntimeWarning: divide by zero encountered in log
  return -np.log(1-image)
/home/david/Institut/rydanalysis/rydanalysis/single_shot/image_processing.py:9: RuntimeWarning: invalid value encountered in log
  return -np.log(1-image)
1020it [00:08, 119.50it/s]


In [5]:
image_list = np.asarray([shot.image for shot in seq.iter_single_shots()])


dod =  xr.DataArray(np.asarray([shot.optical_density[0] for shot in seq.iter_single_shots()])
                    ,dims=('tmstp','pixrow','pixel'),coords={'tmstp':seq.variables.index})

di = xr.DataArray(image_list,dims=('tmstp','image','pixrow','pixel'),coords={'tmstp':seq.variables.index,
                                                            'image':['0','absorbed','2','ref','4','bg']})

dv = xr.DataArray(seq.variables,dims=('tmstp','quantity'))
ds = xr.Dataset(seq.variables)
ds['od']=dod

In [13]:
%%time
ds.to_netcdf('ds.nc')

CPU times: user 98.4 ms, sys: 309 ms, total: 407 ms
Wall time: 789 ms


In [14]:
%%time 
ds = xr.open_dataset('ds.nc')

CPU times: user 6.56 ms, sys: 0 ns, total: 6.56 ms
Wall time: 5.83 ms


In [6]:
var = 'dDDSred'

od_av = ds.groupby(var).mean(dim='tmstp')['od']

In [7]:
od_av

<xarray.DataArray 'od' (dDDSred: 51, pixrow: 80, pixel: 370)>
array([[[ 0.010651, -0.064654, ...,  0.157974, -0.081644],
        [-0.06554 , -0.030739, ..., -0.086199,  0.012077],
        ...,
        [ 0.030367, -0.001374, ...,  0.026784,  0.067027],
        [ 0.063432, -0.105221, ...,  0.02793 , -0.173817]],

       [[-0.130937, -0.135193, ..., -0.006613,  0.010676],
        [-0.035362, -0.099706, ..., -0.137234, -0.000546],
        ...,
        [ 0.081939, -0.012172, ..., -0.164604,  0.053356],
        [-0.079627, -0.137671, ..., -0.035067,  0.193436]],

       ...,

       [[-0.051711,  0.055053, ..., -0.135748, -0.019608],
        [ 0.042468, -0.01331 , ..., -0.110337,  0.062232],
        ...,
        [-0.022116, -0.02303 , ..., -0.025569, -0.191631],
        [ 0.174222,  0.049282, ...,  0.081114,  0.021354]],

       [[ 0.142306, -0.06183 , ...,  0.056041,  0.058945],
        [-0.017494, -0.047544, ..., -0.018274, -0.091951],
        ...,
        [-0.042971,  0.068727, ...,  0.02

In [8]:
od_av.groupby('dDDSred').apply(fit2dGaussian)

<xarray.DataArray (dDDSred: 51)>
array([<rydanalysis.fitting.fitting_2d.fitting2d.Fit2dGaussian object at 0x7f9784610630>,
      dtype=object)
Coordinates:
  * dDDSred  (dDDSred) float64 -4.0 -3.84 -3.68 -3.52 ... 3.52 3.68 3.84 4.0